### Prepare Dataset

<p>
<a href='http://www.paulgraham.com/articles.html' target='_blank'>Paul Graham essays</a>
</p>

In [78]:
# import requests
# import bs4
# import json

# link = 'http://www.aaronsw.com/2002/feeds/pgessays.rss'
# res = requests.get(link)
# sp = bs4.BeautifulSoup(res.text)

# es_links = [[ele for ele in blck.get_text().split('\n') if ele][0]  for blck in sp.find_all('item') ]
# print('total essays:', len(es_links))

# def get_txt(link):
#     try:
#         res = requests.get(link)
#         sp = bs4.BeautifulSoup(res.text)
#         for br in sp('br'):
#             br.replace_with('\n')
#         return sp.find('body').find('table').find('font').get_text()
#     except:
#         print(f'failed: {link}')
# es_txts = [get_txt(link) for link in es_links]

# with open('./paulgraham_essays.json', 'w') as dest:
#     dest.write(json.dumps([ele for ele in es_txts if ele]))

## RNN 

In [79]:
import json
import numpy as np

### Load Dataset

In [194]:
with open('./paulgraham_essays.json', 'r') as f:
    data = json.loads(f.read())

In [195]:
txts = ""
for ele in data:
    txts += ele + '\n'

In [325]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = txts
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 30 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: 
    print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter


data has 1845065 characters, 103 unique.
----
 iD&8*²jhxuD9hZQ;$+và_n
Gz6#⟨xFK("<YGN"!x?éc a1MQ4UIVnt'^?:%Fp!VyeIà<⟨-Fd*'P0R/ eJx9S)L`E]3—f1"18Gy9%3Ki0² [nf⟩≈,T`&N
xi-grs3—0h9oh#"Gh%àF>RS,nAC0mHZ5JV"8tT—(2dg'@fé#_pat
$MQ&MLr4xq0FEI<:VeINx'2AJ5Lm 
----
iter 0, loss: 139.041872
----
 ragh,
slohchatsybq aoothiawnarn em
a t agy hla
lte h
sercoct tb]ia u k tuowAsnoh h yoYoheu ho lynh astx gnedo
tnehoywa h n 
uhtmehfathga ncd h aonea r nbaoygjdu
yershsbty noheothth rshpnfY ,cg I t h o 
----
iter 100, loss: 137.607779
----
  B tiuPruneuyaoml h, töM aAsöavt ao0u sro xat[aad  e taterse ana,pb oaos i tstna Asy Indofo<enotocpt.UwIeihangoloc eu' @ o tsto haoA avre' .er   ou nPe sa,t (eks]tton t bosoa Umroan  nwe tiaorbaikn po 
----
iter 200, loss: 135.220765
----
 hc fe oOcih  nwswatfi
nsh thbnseboesrtrsbhw ]clt )cs   sTo5ebth y
intC r rWoWeaslh  ts lie 
yiuoboloiaw irem sn ustobutg r  d u uoeeotsalpioaeta ms, luraeenudulroiaufecta o oun, u"da,l lucdt stamlhrey 
----
iter 300, loss: 131.589880
-

----
 dod byea,.
F
meld agy
Artho fhotalaloiesmaldes nnogs titosd thapltcaed thing skomly nint
ontotle ind py
xaigg ascaes sgaos ffave of ibis thertesw
it overeticoelk I displenithed nome kirither saveither 
----
iter 3400, loss: 77.741084
----
 udos Hacenenk ant, qofacritd lem thess asocl Cf ass mecwo
lo aagsindisly
soug. Buocomforke inasd a. Tho ereWerily Wo ll mmai ved farididiNin sif binalle
ploed wey wasanm oned alingisidith. deby swco d 
----
iter 3500, loss: 77.250477
----
 n de oul. Toriny ing at iftumea iveens keare
d aveaes wanbtan herwign nhed. _: onkitherle s0neagrenu bped we Roulk as iracnlit es tole eras if vmit mon sas'san
raec's
nous fill efimsin apike de ar,pe 
----
iter 3600, loss: 77.172681
----
  whomapesuncounl. Aden anded pxon. An plhe deato weTheng svees ant ane, od endisgu thinpite ring tired anpul silre orond as ofe wy abs
digne thopllooutimree
yewitoutsiodevi wie'ting luctes ge, lfaytye 
----
iter 3700, loss: 76.974266
----
 on'sen are incicr ingl
Hif pant, A

----
 follenaprens au innakt. [<6 meif thobet,at telet thaod yiontes loto bo lunt
it hemy. On to busut lucbacine rone Abatut che theos.

OS to het
eames ag I grocdhis gunoart sto sorer ath Cfric yithen wade 
----
iter 6900, loss: 70.209810
----
 ertar methe shots rercanmas ye
of sing in sther puoke dint
thast
theveded shot oedes aro jov athe
seertanto the wo lyopp the ae chu the wpinouwn bugy of beoted eagftrictiybers theases gure bo.'d of h 
----
iter 7000, loss: 70.182884
----
  wouissserasile in bien ina irilinxing yvabnomeancourge handy Ie retif p pone theor se
raly sad earders Lthce wins ofd ve yun'osoch , gsithers.

Win
tho.
Ome he ute
mare the havrers. Mnien fou mast pl 
----
iter 7100, loss: 70.045280
----
 sacererte leothit' mitolded seos. Boolu woratire twed klythor on at jotines itsa.

Thoblist in be be.
[Bh thes yxwit the r webie Thhe matere tertoradeld po lid
nat uif

sew. Iud in frow wollinge
the a 
----
iter 7200, loss: 69.758576
----
 itr ofu Cere oltou ;re, for tou si

----
 th yout-bla, nive ryas, the to han alped
arte wase frary want recaving pram riddingicoudpo wanted thicaton. Sou but warliss.

The it quores pro act wheds bpy wh+ho whar juals'r's py scetice pecbo they 
----
iter 10400, loss: 66.244478
----
 ha se ir
have blit the peeamfnaglet,: nat whavy ounaly exalsanpels. I neosy firingato duceoth be, Dreinve of
an? Ang -mor bit doncing prostars juctindide rat of mus shon's be
preabe.


har  he thut th 
----
iter 10500, loss: 66.308455
----
 at the as a demab, oul'c as in exer to nou goreriont. I dovicg th, etha hthric.
Yor mot ary buost a pricide and to do lurgo cors It's muscisiss. Mou ancfe'le greceer is Yah ar. Pfuradt. Brectorlith yo 
----
iter 10600, loss: 66.142662
----
 reensere "Rou bleae.u plean pereatee as Is sumels is at thit mom
ot to so ars ffunt favith sithy ebe of shit handute. Yu. Ifle't reaints
fomoment. As recay becath an'igl ashon to thet to at hande exwa 
----
iter 10700, loss: 66.206913
----
 bactrean harr
and tha
ise teve

----
 ade ty aboow tony funs creswer) a mecas.
hae soulle to bugtirel mo on get prou.  Ssersincha'le to as agennle
sove thinof radiis storake it quens
on
reat of but the mac mot ve.  Iu i uthes mujed to on  
----
iter 13800, loss: 64.824509
----
 orsanven oneathy geoftaliedsithally in could preenseel
wally
are icel the 
that siveacer pethe oulde a Scacaned tado deomed Aare serins in bugho, poots intitomo? An dicer is baweal
omact the reaprenco 
----
iter 13900, loss: 64.860375
----
 dome? 
Near con do teme dow
sees. 
30" as ay wene. 
Nofthing thitiity ionealilly abu
osellstor peopenopchise ifo candilly not meme and tachicuith
multith im we
one the dulat it be want proking.
dou'tb 
----
iter 14000, loss: 64.781027
----
 otrus a c ally't the seave cherte wall ice frakprecwel that peopres at is Peanf whoilienotining hestartutiverinids
remrere deapre the rexpl
ay anchind wome suimitem eserilith If a as fould comruothe p 
----
iter 14100, loss: 64.634789
----
 lll peotretrour "thpy Ythe tro

...

----
 " years you dighifarshangidince
parkent a momes ols
see toming wor tnotent kind comment?

I've just out anyens,
inserval the you will go blingies of tommoureplosher. 
Orston into make of and and is on 
----
iter 690400, loss: 51.189177
----
 
ster
and you matific to empramity out
better was of you're use fockn salts leant your
then hrany boted explesing of you iffered Fold you leveatobation stridery a pan with to at larrant would would ex 
----
iter 690500, loss: 51.004461
----
 hem one icorscajounscees thy companies. Invealiming. The one anselss them.



  Nor  the
catifinive
able we premiting that it reallers likable knever news conveents.  If fitter, Jeativialthor havel i 
----
iter 690600, loss: 50.961194
----
 gon't exwark, to's into, is at to leagt.  You
like go tustlemmising yook at
hely if  dolug inticarything at aranther, you don't why work.
Dequencricy (I wan wal in deal. In acciale you it it the fire  
----
iter 690700, loss: 51.277015
----
 of mon't a naveral phochod

----
 of that get firgry
would corlece the 2)0 seemed the Swast want to no fow sching tauks Reakerd" about those to price oversies, this of ander. Some to
stors a propoble
them.

Jess if it
company
this of  
----
iter 693800, loss: 51.919393
----
  gradis to to stank the vacking.

[3]

`

[1]





An you work.  Sometist to.  Whine everysher they wark and like intttasicind win with hathing ander,
know art is in knicking to cares
same these, beti 
----
iter 693900, loss: 51.698383
----
 .


Mathed 
you hay because.  [10]

Je had a company deved manty the can lowed 14.6%, it dseretinged maky
got a like 2075, Ro trelon" fill for
teat, doxes is may be othn Ox than of taxamb windiverncan 
----
iter 694000, loss: 51.540102
----
 ng.  Imolimalffate's after mowed markient writes usuenion the dore stop to ged caince fou let hook.

Perany, somether, Enolicknelfore not it reatit, most to be efell of capecais they troge.


Wer weer 
----
iter 694100, loss: 51.381252
----
 he simple Lually of comvle

----
 Collent some for the monend go favid
alaging but there probers because working and ith took ander starthing desesive stories ampliming wa prigers.  West will be. AAme  So intrevident
bites you stors,  
----
iter 697200, loss: 54.187891
----
 nt) Well hue so — fachiouslay to sist how of persona wat becauselyr, their threquitice pately" something infuse fix dat
of twroode able to schict orn't
alseebunt a people of nother it kolcanians tenta 
----
iter 697300, loss: 54.101133
----
 which 1/6. 
Ate.

Or most our by mighowh these pretent Weaded to fours, longe.  I
con.

We them esmienthor fow ofdem.
Forf feans.  You're
dillohis people roundre readen
simp peoplet the toreen't enoug 
----
iter 697400, loss: 53.752566
----
 nce, is condeldming schoinaso
succianed cererd
I destorasents the some tealegial to a canohon out mover to
will be sy dol.

This
tes to ones, mach they are 2019
Rofthors is.  There's a Antial peram th 
----
iter 697500, loss: 53.601817
----
  sure the
maning to want m

----
 rstride—
noth what ith with you'd best om
loted hcound Jade reas ald inforgatios, that Coslig. Thein ront entigay, jubst it out
is: ofttabert curce doom, and that of adstegn ithe
I lition times had is 
----
iter 700600, loss: 51.116800
----
 at of
to problem, real chery kingribloun of that might produps a ways.  In the prolit to you get mibuall is seendrrarks thy prarificforicion ploss write stalke
are lead wherdels towingly
to fromeation 
----
iter 700700, loss: 51.490510
----
 ndide, somery a someone can emb for to ingstal, expertarer evily the Eopecteves oned cart to you 1969 in quesont.  Renolve to dy ith conthing. 
Them vow to by then one. And not most to atren uy, in le 
----
iter 700800, loss: 51.695790
----
 dick thints.

Subliciress, bpough
who have it scortseticchend
think Isko 8Quter dead toon in is are beat?

I tethed do grady.  Reakity aldors.  It inpelf-signioned may, some on of the
intimess they ca 
----
iter 700900, loss: 51.686490
----
 o up true
roftwarly fout o

KeyboardInterrupt: 